In [105]:
#Loading the data set - training data.
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from gensim.models import KeyedVectors
from sklearn.preprocessing import Imputer
from nltk.tokenize import wordpunct_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegressionCV
from sklearn.svm import LinearSVC, NuSVC
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [132]:
df=pd.read_csv('reviews.tsv',sep='\t', encoding='utf-8',names = ['rating','review_id','user_id', 'book_id','review'])
print(len(df))
df.rating[df.rating == 2] = 1
df.rating[df.rating == 4] = 5
df.rating.value_counts()

63257


C:\Users\aas3n17\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\aas3n17\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


5    42832
3    12201
1     8224
Name: rating, dtype: int64

In [134]:
df.rating[df.rating == 1] = 0
df.rating.value_counts()

C:\Users\aas3n17\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


5    42832
3    12201
0     8224
Name: rating, dtype: int64

In [135]:
df.rating[df.rating == 5] = 1
df.rating.value_counts()

C:\Users\aas3n17\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


1    42832
3    12201
0     8224
Name: rating, dtype: int64

In [137]:
neg=df[df.rating == 0] 
pos=df[df.rating == 1] 

In [140]:
neg=df[df.rating == 0] 
pos=df[df.rating == 1] 
test_neg=neg[0:1500]
train_neg=neg[1500:8224]
test_pos=pos[0:1500]
train_pos=pos[1500:8224]
train=pd.concat([train_neg, train_pos])
test=pd.concat([test_neg, test_pos])
from sklearn.utils import shuffle
train = shuffle(train)
test = shuffle(test)

In [141]:
train=pd.concat([train_neg, train_pos])
test=pd.concat([test_neg, test_pos])

In [151]:
from sklearn.utils import shuffle
train = shuffle(train)
test = shuffle(test)

In [152]:
train.rating.value_counts()

1    6724
0    6724
Name: rating, dtype: int64

In [154]:
test.rating.value_counts()

1    1500
0    1500
Name: rating, dtype: int64

In [145]:
len(train)

13448

In [155]:
len(test)

3000

In [175]:
reviews= clean_raw_review( test.review)

TypeError: expected string or bytes-like object

In [180]:
cleaned_rev=[]
for review in test:
    cleaned_rev.append(clean_raw_review(review))

AttributeError: 'str' object has no attribute 'empty'

In [178]:
def arabicrange():
        """return a list of arabic characteres .
        Return a list of characteres between \u060c to \u0652
        @return: list of arabic characteres.
        @rtype: unicode;
        """
        mylist=[];
        for i in range(0x0600, 0x00653):
            try :
                mylist.append(chr(i));
            except ValueError:
                pass;
        return mylist;

    # cleans a single review

In [171]:
    import codecs
    import numpy as np
    import pandas as pd
    import re
    def clean_raw_review( body):
         # patterns to remove first
        pat = [\
            (u'http[s]?://[a-zA-Z0-9_\-./~\?=%&]+', u''),               # remove links
            (u'www[a-zA-Z0-9_\-?=%&/.~]+', u''),
#            u'\n+': u' ',                     # remove newlines
            (u'<br />', u' '),                  # remove html line breaks
            (u'</?[^>]+>', u' '),              # remove html markup
#            u'http': u'',
            (u'[a-zA-Z]+\.org', u''),
            (u'[a-zA-Z]+\.com', u''),
            (u'://', u''),
            (u'&[^;]+;', u' '),
            (u':D', u':)'),
#            (u'[0-9/]+', u''),
#            u'[a-zA-Z.]+': u'',
#            u'[^0-9' + u''.join(self.arabicrange()) + \
#                u"!.,;:$%&*%'#(){}~`\[\]/\\\\\"" + \
#                u'\s^><\-_\u201D\u00AB=\u2026]+': u'',          # remove latin characters
            (u'\s+', u' '),                     # remove spaces
            (u'\.+', u'.'),                     # multiple dots
            (u'[\u201C\u201D]', u'"'),          #“
            (u'[\u2665\u2764]', u''),           # heart symbol
            (u'[\u00BB\u00AB]', u'"'),
            (u'\u2013', u'-'),                # dash
        ]

        # patterns that disqualify a review
        remove_if_there = [\
            (u'[^0-9' + u''.join(arabicrange()) + \
                u"!.,;:$%&*%'#(){}~`\[\]/\\\\\"" + \
                u'\s\^><\-_\u201D\u00AB=\u2026+|' + \
                u'\u0660-\u066D\u201C\u201D' + \
                u'\ufefb\ufef7\ufef5\ufef9]+', u''),                   # non arabic characters
        ]

        # patterns that disqualify if empty after removing
        remove_if_empty_after = [\
            (u'[0-9a-zA-Z\-_]', u' '),             #alpha-numeric
            (u'[0-9' + u".,!;:$%&*%'#(){}~`\[\]/\\\\\"" + \
                u'\s\^><`\-=_+]+', u''),                  # remove just punctuation
            (u'\s+', u' '),                     # remove spaces
        ]

        # remove again
        # patterns to remove
        pat2 = [\
#            u'[^0-9' + u''.join(self.arabicrange()) + \
#                u"!.,;:$%&*%'#(){}~`\[\]/\\\\\"" + \
#                u'\s^><\-_\u201D\u00AB=\u2026]+': u'',          # remove latin characters
        ]

        skip = False



        # do some subsitutions
        for k,v in pat:
            body.empty = re.sub(k, v, body)

        # remove if exist
        for k,v in remove_if_there:
            if re.search(k, body):
                skip = True

        # remove if empty after replacing
        for k,v in remove_if_empty_after:
            temp = re.sub(k, v, body)
            if not temp.empty:
                skip = True

        # do some more subsitutions
        if not skip:
            for k,v in pat2:
                body = re.sub(k, v, body)

        # if empty string, skip
        if body.empty:
            skip = True

        if not skip:
            return body
        else:
            return ""

    # Read raw reviews from file and clean and write into clean_reviews
    

    

In [99]:
    def clean(df):
        path = "Arabic_stop_word.txt"
        stop_words = []
        with codecs.open(path, "r", encoding="utf-8", errors="ignore") as myfile:
            stop_words = myfile.readlines()
        stop_words = [word.strip() for word in stop_words]
        arabic_punctuations = '''؛<>_()*،&^%][ـ،/:"؟ـ`÷×.,'{}~¦+|!”…“–_'''
        english_punctuations = string.punctuation
        punctuations_list = arabic_punctuations + english_punctuations

        arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
        for comments in df:
            
            
            # removearabic_diacritics
            df['txt'] = df['txt'].map(lambda x: re.sub(arabic_diacritics,'',str(x)))

            #def normalize_arabic(text)
            df['txt'] = df['txt'].map(lambda x: re.sub("[إأآا]", "ا",str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("ؤ", "ء",str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("ى", "ي",str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("ئ", "ء",str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("ة", "ه",str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("گ", "ك",str(x)))

            

            #def remove_punctuations(text):
            df['txt'] = df['txt'].map(lambda x: re.sub("["+punctuations_list+"]",'',str(x)))

            # remove_repeating_char(text):
            df['txt'] = df['txt'].map(lambda x: re.sub(r'(.)\1+', r'\1', str(x)))
            # remove '\\n'
            df['txt'] = df['txt'].map(lambda x: re.sub('\\n',' ',str(x)))

            # remove any text starting with User... 
            df['txt'] = df['txt'].map(lambda x: re.sub("\[\[User.*",'',str(x)))

            # remove IP addresses or user IDs
            df['txt'] = df['txt'].map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))

            # lower uppercase letters
            #df['txt'] = df['txt'].map(lambda x: str(x).lower())

            #remove http links in the text
            df['txt'] = df['txt'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))

            #remove all punctuation
            df['txt'] = df['txt'].map(lambda x: re.sub("_", '',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("«", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("»", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("“", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("”", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("♥", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("♡", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("¥", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("؟", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("!", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("#", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("$", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("%", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("&", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("✖", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('ة', 'ه',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub(' ئ', 'ء',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('ؤ', 'ء',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('ے', 'ك',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('ڪ', 'ك',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('أ', 'ا',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('إ', 'ا',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('آ', 'ا',str(x)))

            df['txt']= df['txt'].map(lambda x: re.sub("["+string.punctuation+"]",'',str(x)))
            df['txt']= df['txt'].map(lambda x: re.sub("["+punctuations_list+"]",'',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("["+digits+"]",'',str(x)))

            #df['txt'] = df['txt'].map(lambda x: re.sub(r'[^\x600-\x6ff]','',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub(r'[a-zA-Z?]','',str(x))) 
            df['txt'].apply(lambda x: [item for item in x if item not in stop_words])
        return df